# Módulo 2: Introducción a Llama 3.1

## Demostración práctica de las capacidades y características de Llama 3.1

**Autor:** Tomas Vera  
**Fecha:** Julio 2025

---

En este notebook exploraremos las capacidades de Llama 3.1, incluyendo:

1. **Carga del modelo** - Configuración básica y cuantizada
2. **Capacidades básicas** - Generación de texto general
3. **Capacidades conversacionales** - Formato de chat
4. **Capacidades multilingües** - Soporte para múltiples idiomas
5. **Generación de código** - Programación asistida por IA
6. **Análisis de rendimiento** - Métricas y optimización
7. **Comparación de configuraciones** - Diferentes parámetros de generación

## 1. Importaciones y Configuración Inicial

In [1]:
#!/usr/bin/env python3
import torch
import time
import psutil
import os
import sys
from pathlib import Path

# Agregar el directorio raíz al path para importar config
sys.path.append(str(Path.cwd().parent))

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from typing import List, Dict, Optional
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Importar configuración
try:
    from config.config_loader import setup_huggingface_auth, get_model_config
    HAS_CONFIG = True
    print("✅ Token HF cargado desde hf_config.py")
except ImportError:
    print("⚠️ No se pudo cargar la configuración. Usando configuración por defecto.")
    HAS_CONFIG = False

print("📦 Librerías importadas correctamente")

2025-08-01 00:24:38.570708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754007878.880280      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754007878.965640      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


⚠️ No se pudo cargar la configuración. Usando configuración por defecto.
📦 Librerías importadas correctamente


## 2. Definición de la Clase Llama31Demo

In [2]:
class Llama31Demo:
    """Clase para demostrar las capacidades de Llama 3.1"""
    
    def __init__(self, model_name: str = None):
        # Configurar autenticación si está disponible
        if HAS_CONFIG:
            setup_huggingface_auth()
            config = get_model_config()
            self.model_name = model_name or config['default_model']
        else:
            self.model_name = model_name or "meta-llama/Meta-Llama-3.1-8B-Instruct"

        self.tokenizer = None
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🦙 Inicializando demostración de Llama 3.1")
        print(f"📱 Dispositivo: {self.device}")
        print(f"🤖 Modelo: {self.model_name}")
    
    def _obtener_uso_memoria(self) -> float:
        """Obtener uso actual de memoria en MB"""
        process = psutil.Process(os.getpid())
        return process.memory_info().rss / (1024 * 1024)
    
    def _mostrar_info_modelo(self):
        """Mostrar información detallada del modelo"""
        if self.model is None:
            return
        
        # Contar parámetros
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        
        print(f"\n📊 Información del modelo:")
        print(f"   Nombre: {self.model_name}")
        print(f"   Parámetros totales: {total_params:,}")
        print(f"   Parámetros entrenables: {trainable_params:,}")
        print(f"   Dispositivo: {next(self.model.parameters()).device}")
        print(f"   Tipo de datos: {next(self.model.parameters()).dtype}")
        
        # Uso de memoria
        memoria_mb = self._obtener_uso_memoria()
        print(f"   Memoria utilizada: {memoria_mb:.2f} MB")

# Crear instancia de la demo
demo = Llama31Demo()
print("✅ Instancia de Llama31Demo creada")

🦙 Inicializando demostración de Llama 3.1
📱 Dispositivo: cpu
🤖 Modelo: meta-llama/Meta-Llama-3.1-8B-Instruct
✅ Instancia de Llama31Demo creada


## 3. Carga del Modelo - Configuración Cuantizada

Primero intentaremos cargar el modelo con cuantización 4-bit para optimizar el uso de memoria.

In [3]:
def cargar_modelo_cuantizado(self):
    """Cargar Llama 3.1 con cuantización para eficiencia"""
    print("\n" + "="*60)
    print("⚡ CARGANDO LLAMA 3.1 - CONFIGURACIÓN CUANTIZADA")
    print("="*60)
    
    try:
        # Configuración de cuantización
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        
        print("🔧 Configuración de cuantización 4-bit activada")
        
        # Cargar tokenizer
        if self.tokenizer is None:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                token=True if HAS_CONFIG else None
            )
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Cargar modelo cuantizado
        print("🧠 Cargando modelo cuantizado...")
        start_time = time.time()
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            token=True if HAS_CONFIG else None
        )
        
        load_time = time.time() - start_time
        
        print(f"✅ Modelo cuantizado cargado en {load_time:.2f} segundos")
        print("💾 Uso de memoria significativamente reducido")
        
        # Mostrar información
        self._mostrar_info_modelo()
        
    except Exception as e:
        print(f"❌ Error cargando modelo cuantizado: {e}")
        if "bitsandbytes" in str(e):
            print("💡 Instalación requerida: pip install bitsandbytes")
        print("🔄 Cargando modelo básico...")
        return False
    return True

# Agregar método a la instancia
Llama31Demo.cargar_modelo_cuantizado = cargar_modelo_cuantizado

# Intentar cargar modelo cuantizado
print("🔄 Intentando cargar modelo cuantizado...")
exito_cuantizado = demo.cargar_modelo_cuantizado()

🔄 Intentando cargar modelo cuantizado...

⚡ CARGANDO LLAMA 3.1 - CONFIGURACIÓN CUANTIZADA
❌ Error cargando modelo cuantizado: No package metadata was found for bitsandbytes
💡 Instalación requerida: pip install bitsandbytes
🔄 Cargando modelo básico...


## 4. Carga del Modelo - Configuración Básica

Si la cuantización falla, cargaremos el modelo en configuración básica.

In [4]:
def cargar_modelo_basico(self):
    """Cargar Llama 3.1 en configuración básica"""
    print("\n" + "="*60)
    print("📥 CARGANDO LLAMA 3.1 - CONFIGURACIÓN BÁSICA")
    print("="*60)
    
    try:
        # Cargar tokenizer
        print("🔤 Cargando tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=True if HAS_CONFIG else None
        )
        
        # Configurar pad token si no existe
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        print(f"✅ Tokenizer cargado")
        print(f"   Vocabulario: {self.tokenizer.vocab_size:,} tokens")
        print(f"   Tokens especiales: {len(self.tokenizer.special_tokens_map)}")
        
        # Cargar modelo
        print("🧠 Cargando modelo...")
        start_time = time.time()
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else "cpu",
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            token=True if HAS_CONFIG else None
        )
        
        load_time = time.time() - start_time
        
        print(f"✅ Modelo cargado en {load_time:.2f} segundos")
        
        # Información del modelo
        self._mostrar_info_modelo()
        
    except Exception as e:
        print(f"❌ Error cargando modelo: {e}")
        print("💡 Intentando con modelo alternativo...")

        # Intentar con modelo alternativo
        try:
            fallback_model = "microsoft/DialoGPT-medium"
            print(f"🔄 Cargando modelo alternativo: {fallback_model}")

            self.tokenizer = AutoTokenizer.from_pretrained(fallback_model)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            self.model = AutoModelForCausalLM.from_pretrained(
                fallback_model,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else "cpu",
                low_cpu_mem_usage=True
            )

            self.model_name = fallback_model
            print(f"✅ Modelo alternativo cargado exitosamente")
            self._mostrar_info_modelo()

        except Exception as fallback_error:
            print(f"❌ Error con modelo alternativo: {fallback_error}")
            print("💡 Sugerencias:")
            print("   - Verifica que tengas acceso al modelo")
            print("   - Asegúrate de tener suficiente memoria RAM")
            print("   - Considera usar cuantización para reducir memoria")

# Agregar método a la instancia
Llama31Demo.cargar_modelo_basico = cargar_modelo_basico

# Si no se cargó el modelo cuantizado, cargar el básico
if not exito_cuantizado or demo.model is None:
    print("🔄 Cargando modelo básico...")
    demo.cargar_modelo_basico()

if demo.model is None:
    print("❌ No se pudo cargar el modelo. Verifica:")
    print("   - Conexión a internet")
    print("   - Acceso al modelo de Hugging Face")
    print("   - Memoria RAM suficiente")
else:
    print("\n🎉 ¡Modelo cargado exitosamente!")

🔄 Cargando modelo básico...

📥 CARGANDO LLAMA 3.1 - CONFIGURACIÓN BÁSICA
🔤 Cargando tokenizer...
❌ Error cargando modelo: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-688c095b-14e4d5823f3e7e9227fa32d3;05fa0266-2f81-41a9-8aa6-07b115216f09)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.
💡 Intentando con modelo alternativo...
🔄 Cargando modelo alternativo: microsoft/DialoGPT-medium


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Modelo alternativo cargado exitosamente

📊 Información del modelo:
   Nombre: microsoft/DialoGPT-medium
   Parámetros totales: 354,823,168
   Parámetros entrenables: 354,823,168
   Dispositivo: cpu
   Tipo de datos: torch.float32
   Memoria utilizada: 3918.53 MB

🎉 ¡Modelo cargado exitosamente!


## 5. Funciones de Generación de Texto

In [5]:
def _generar_respuesta(self, prompt: str, max_tokens: int = 100, **kwargs) -> str:
    """Generar respuesta para un prompt dado"""
    try:
        # Configuración por defecto
        config_default = {
            "max_new_tokens": max_tokens,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": self.tokenizer.eos_token_id
        }
        
        # Actualizar con parámetros personalizados
        config_default.update(kwargs)
        
        # Tokenizar entrada
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        
        # Mover a dispositivo correcto
        if self.device == "cuda":
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Generar
        with torch.no_grad():
            outputs = self.model.generate(**inputs, **config_default)
        
        # Decodificar solo los tokens nuevos
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return response.strip()
        
    except Exception as e:
        return f"Error generando respuesta: {e}"

def _generar_respuesta_chat(self, conversacion: List[Dict]) -> str:
    """Generar respuesta usando formato de chat"""
    try:
        # Aplicar chat template
        prompt = self.tokenizer.apply_chat_template(
            conversacion, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        return self._generar_respuesta(prompt, max_tokens=150)
        
    except Exception as e:
        return f"Error en chat: {e}"

def _mostrar_conversacion(self, conversacion: List[Dict]):
    """Mostrar conversación de forma legible"""
    for mensaje in conversacion:
        rol = mensaje["role"]
        contenido = mensaje["content"]
        
        if rol == "system":
            print(f"🔧 Sistema: {contenido}")
        elif rol == "user":
            print(f"👤 Usuario: {contenido}")
        elif rol == "assistant":
            print(f"🤖 Asistente: {contenido}")

# Agregar métodos a la instancia
Llama31Demo._generar_respuesta = _generar_respuesta
Llama31Demo._generar_respuesta_chat = _generar_respuesta_chat
Llama31Demo._mostrar_conversacion = _mostrar_conversacion

print("✅ Funciones de generación agregadas")

✅ Funciones de generación agregadas


## 6. Demostración de Capacidades Básicas

Vamos a probar las capacidades básicas de generación de texto con diferentes tipos de prompts.

In [6]:
def demo_capacidades_basicas(self):
    """Demostrar capacidades básicas de generación"""
    print("\n" + "="*60)
    print("🎯 DEMOSTRACIÓN DE CAPACIDADES BÁSICAS")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado. Ejecuta cargar_modelo_basico() primero.")
        return
    
    # Ejemplos de prompts
    prompts = [
        "Explica qué es la inteligencia artificial en términos simples:",
        "Escribe un código Python para calcular números primos:",
        "¿Cuáles son los beneficios de la energía renovable?",
        "Traduce al inglés: 'La tecnología está cambiando el mundo'",
        "Resuelve: Si tengo 15 manzanas y como 3, ¿cuántas me quedan?"
    ]
    
    for i, prompt in enumerate(prompts, 1):
        print(f"\n🔍 Ejemplo {i}:")
        print(f"Prompt: {prompt}")
        
        # Generar respuesta
        respuesta = self._generar_respuesta(prompt, max_tokens=100)
        print(f"Respuesta: {respuesta}")
        print("-" * 40)

# Agregar método a la instancia
Llama31Demo.demo_capacidades_basicas = demo_capacidades_basicas

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_basicas()
else:
    print("⚠️ Modelo no cargado. Saltando demostración de capacidades básicas.")


🎯 DEMOSTRACIÓN DE CAPACIDADES BÁSICAS

🔍 Ejemplo 1:
Prompt: Explica qué es la inteligencia artificial en términos simples:
Respuesta: el que vous avez.
----------------------------------------

🔍 Ejemplo 2:
Prompt: Escribe un código Python para calcular números primos:
Respuesta: 3
----------------------------------------

🔍 Ejemplo 3:
Prompt: ¿Cuáles son los beneficios de la energía renovable?
Respuesta: 
----------------------------------------

🔍 Ejemplo 4:
Prompt: Traduce al inglés: 'La tecnología está cambiando el mundo'
Respuesta: a a cambiado.
----------------------------------------

🔍 Ejemplo 5:
Prompt: Resuelve: Si tengo 15 manzanas y como 3, ¿cuántas me quedan?
Respuesta: 
----------------------------------------


## 7. Demostración de Capacidades Conversacionales

Probemos las capacidades de chat del modelo usando el formato de conversación.

In [7]:
def demo_capacidades_conversacionales(self):
    """Demostrar capacidades conversacionales con formato de chat"""
    print("\n" + "="*60)
    print("💬 DEMOSTRACIÓN DE CAPACIDADES CONVERSACIONALES")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Conversación de ejemplo
    conversacion = [
        {
            "role": "system",
            "content": "Eres un asistente útil y amigable que responde de manera clara y concisa."
        },
        {
            "role": "user", 
            "content": "Hola, ¿puedes explicarme qué es machine learning?"
        }
    ]
    
    print("🗣️ Conversación de ejemplo:")
    self._mostrar_conversacion(conversacion)
    
    # Generar respuesta usando chat template
    respuesta = self._generar_respuesta_chat(conversacion)
    
    conversacion.append({
        "role": "assistant",
        "content": respuesta
    })
    
    print(f"🤖 Asistente: {respuesta}")
    
    # Continuar conversación
    conversacion.append({
        "role": "user",
        "content": "¿Puedes darme un ejemplo práctico?"
    })
    
    print(f"\n👤 Usuario: ¿Puedes darme un ejemplo práctico?")
    
    respuesta2 = self._generar_respuesta_chat(conversacion)
    print(f"🤖 Asistente: {respuesta2}")

# Agregar método a la instancia
Llama31Demo.demo_capacidades_conversacionales = demo_capacidades_conversacionales

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_conversacionales()
else:
    print("⚠️ Modelo no cargado. Saltando demostración conversacional.")


💬 DEMOSTRACIÓN DE CAPACIDADES CONVERSACIONALES
🗣️ Conversación de ejemplo:
🔧 Sistema: Eres un asistente útil y amigable que responde de manera clara y concisa.
👤 Usuario: Hola, ¿puedes explicarme qué es machine learning?
🤖 Asistente: No se.

👤 Usuario: ¿Puedes darme un ejemplo práctico?
🤖 Asistente: No soy de acuerdo


## 8. Demostración de Capacidades Multilingües

Exploremos las capacidades del modelo en diferentes idiomas.

In [8]:
def demo_capacidades_multilingues(self):
    """Demostrar capacidades multilingües"""
    print("\n" + "="*60)
    print("🌍 DEMOSTRACIÓN DE CAPACIDADES MULTILINGÜES")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Prompts en diferentes idiomas
    prompts_multilingues = {
        "Español": "Describe las ventajas de la energía solar:",
        "English": "Explain the benefits of renewable energy:",
        "Français": "Expliquez les avantages de l'énergie solaire:",
        "Deutsch": "Erklären Sie die Vorteile der Solarenergie:",
        "Italiano": "Spiega i vantaggi dell'energia solare:",
        "Português": "Explique as vantagens da energia solar:"
    }
    
    for idioma, prompt in prompts_multilingues.items():
        print(f"\n🗣️ {idioma}:")
        print(f"Prompt: {prompt}")
        
        respuesta = self._generar_respuesta(prompt, max_tokens=80)
        print(f"Respuesta: {respuesta}")
        print("-" * 40)

# Agregar método a la instancia
Llama31Demo.demo_capacidades_multilingues = demo_capacidades_multilingues

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_multilingues()
else:
    print("⚠️ Modelo no cargado. Saltando demostración multilingüe.")


🌍 DEMOSTRACIÓN DE CAPACIDADES MULTILINGÜES

🗣️ Español:
Prompt: Describe las ventajas de la energía solar:
Respuesta: a verde el a cual
----------------------------------------

🗣️ English:
Prompt: Explain the benefits of renewable energy:
Respuesta: a lot of the energy comes from renewable energy sources.
----------------------------------------

🗣️ Français:
Prompt: Expliquez les avantages de l'énergie solaire:
Respuesta: en France, ils est un peu d'appeler avec une voix de la vie.
----------------------------------------

🗣️ Deutsch:
Prompt: Erklären Sie die Vorteile der Solarenergie:
Respuesta: P
----------------------------------------

🗣️ Italiano:
Prompt: Spiega i vantaggi dell'energia solare:
Respuesta: 
----------------------------------------

🗣️ Português:
Prompt: Explique as vantagens da energia solar:
Respuesta: P
----------------------------------------


## 9. Demostración de Capacidades de Código

Probemos las capacidades del modelo para generar código en diferentes lenguajes de programación.

In [9]:
def demo_capacidades_codigo(self):
    """Demostrar capacidades de generación de código"""
    print("\n" + "="*60)
    print("💻 DEMOSTRACIÓN DE CAPACIDADES DE CÓDIGO")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Prompts de programación
    prompts_codigo = [
        "Escribe una función Python para calcular el factorial de un número:",
        "Crea una clase JavaScript para manejar una lista de tareas:",
        "Escribe una consulta SQL para obtener los 10 productos más vendidos:",
        "Implementa un algoritmo de búsqueda binaria en Python:"
    ]
    
    for i, prompt in enumerate(prompts_codigo, 1):
        print(f"\n💻 Ejemplo de código {i}:")
        print(f"Prompt: {prompt}")
        
        respuesta = self._generar_respuesta(prompt, max_tokens=200)
        print(f"Código generado:\n{respuesta}")
        print("-" * 50)

# Agregar método a la instancia
Llama31Demo.demo_capacidades_codigo = demo_capacidades_codigo

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_capacidades_codigo()
else:
    print("⚠️ Modelo no cargado. Saltando demostración de código.")


💻 DEMOSTRACIÓN DE CAPACIDADES DE CÓDIGO

💻 Ejemplo de código 1:
Prompt: Escribe una función Python para calcular el factorial de un número:
Código generado:
P
--------------------------------------------------

💻 Ejemplo de código 2:
Prompt: Crea una clase JavaScript para manejar una lista de tareas:
Código generado:

--------------------------------------------------

💻 Ejemplo de código 3:
Prompt: Escribe una consulta SQL para obtener los 10 productos más vendidos:
Código generado:
P
--------------------------------------------------

💻 Ejemplo de código 4:
Prompt: Implementa un algoritmo de búsqueda binaria en Python:
Código generado:
P
--------------------------------------------------


## 10. Análisis de Rendimiento

Analicemos el rendimiento del modelo midiendo tiempo de respuesta y uso de memoria.

In [10]:
def demo_analisis_rendimiento(self):
    """Analizar rendimiento del modelo"""
    print("\n" + "="*60)
    print("📊 ANÁLISIS DE RENDIMIENTO")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    # Test de rendimiento
    prompt_test = "Explica el concepto de inteligencia artificial y sus aplicaciones en la industria moderna:"
    
    print(f"🧪 Prompt de prueba: {prompt_test}")
    
    # Medir tiempo y memoria
    memoria_inicial = self._obtener_uso_memoria()
    
    tiempos = []
    longitudes = []
    
    for i in range(3):
        print(f"\n🔄 Ejecución {i+1}/3:")
        
        start_time = time.time()
        respuesta = self._generar_respuesta(prompt_test, max_tokens=150)
        end_time = time.time()
        
        tiempo_generacion = end_time - start_time
        longitud_respuesta = len(respuesta.split())
        
        tiempos.append(tiempo_generacion)
        longitudes.append(longitud_respuesta)
        
        tokens_por_segundo = longitud_respuesta / tiempo_generacion if tiempo_generacion > 0 else 0
        
        print(f"   Tiempo: {tiempo_generacion:.2f}s")
        print(f"   Palabras generadas: {longitud_respuesta}")
        print(f"   Velocidad: {tokens_por_segundo:.2f} palabras/s")
    
    memoria_final = self._obtener_uso_memoria()
    
    # Estadísticas finales
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"   Tiempo promedio: {sum(tiempos)/len(tiempos):.2f}s")
    print(f"   Velocidad promedio: {sum(longitudes)/sum(tiempos):.2f} palabras/s")
    print(f"   Uso de memoria: {memoria_final - memoria_inicial:.2f} MB")
    print(f"   Memoria total usada: {memoria_final:.2f} MB")

# Agregar método a la instancia
Llama31Demo.demo_analisis_rendimiento = demo_analisis_rendimiento

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_analisis_rendimiento()
else:
    print("⚠️ Modelo no cargado. Saltando análisis de rendimiento.")


📊 ANÁLISIS DE RENDIMIENTO
🧪 Prompt de prueba: Explica el concepto de inteligencia artificial y sus aplicaciones en la industria moderna:

🔄 Ejecución 1/3:
   Tiempo: 0.33s
   Palabras generadas: 0
   Velocidad: 0.00 palabras/s

🔄 Ejecución 2/3:
   Tiempo: 0.43s
   Palabras generadas: 1
   Velocidad: 2.35 palabras/s

🔄 Ejecución 3/3:
   Tiempo: 0.43s
   Palabras generadas: 1
   Velocidad: 2.34 palabras/s

📈 ESTADÍSTICAS FINALES:
   Tiempo promedio: 0.39s
   Velocidad promedio: 1.69 palabras/s
   Uso de memoria: 0.00 MB
   Memoria total usada: 3415.94 MB


## 11. Comparación de Configuraciones de Generación

Comparemos diferentes configuraciones de generación para ver cómo afectan la creatividad y coherencia del modelo.

In [11]:
def demo_comparacion_configuraciones(self):
    """Comparar diferentes configuraciones de generación"""
    print("\n" + "="*60)
    print("⚙️ COMPARACIÓN DE CONFIGURACIONES")
    print("="*60)
    
    if self.model is None or self.tokenizer is None:
        print("❌ Modelo no cargado.")
        return
    
    prompt = "Escribe un párrafo sobre el futuro de la tecnología:"
    
    configuraciones = {
        "Conservadora": {"temperature": 0.3, "top_p": 0.8, "do_sample": True},
        "Balanceada": {"temperature": 0.7, "top_p": 0.9, "do_sample": True},
        "Creativa": {"temperature": 1.0, "top_p": 0.95, "do_sample": True},
        "Determinística": {"temperature": 0.0, "do_sample": False}
    }
    
    print(f"🎯 Prompt: {prompt}")
    
    for nombre, config in configuraciones.items():
        print(f"\n🔧 Configuración {nombre}:")
        print(f"   Parámetros: {config}")
        
        respuesta = self._generar_respuesta(prompt, max_tokens=100, **config)
        print(f"   Respuesta: {respuesta}")
        print("-" * 50)

# Agregar método a la instancia
Llama31Demo.demo_comparacion_configuraciones = demo_comparacion_configuraciones

# Ejecutar demostración si el modelo está cargado
if demo.model is not None:
    demo.demo_comparacion_configuraciones()
else:
    print("⚠️ Modelo no cargado. Saltando comparación de configuraciones.")


⚙️ COMPARACIÓN DE CONFIGURACIONES
🎯 Prompt: Escribe un párrafo sobre el futuro de la tecnología:

🔧 Configuración Conservadora:
   Parámetros: {'temperature': 0.3, 'top_p': 0.8, 'do_sample': True}
   Respuesta: P
--------------------------------------------------

🔧 Configuración Balanceada:
   Parámetros: {'temperature': 0.7, 'top_p': 0.9, 'do_sample': True}
   Respuesta: 
--------------------------------------------------

🔧 Configuración Creativa:
   Parámetros: {'temperature': 1.0, 'top_p': 0.95, 'do_sample': True}


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   Respuesta: P
--------------------------------------------------

🔧 Configuración Determinística:
   Parámetros: {'temperature': 0.0, 'do_sample': False}
   Respuesta: P
--------------------------------------------------


## 12. Resumen y Conclusiones

¡Felicidades! Has completado la demostración de Llama 3.1. En este notebook hemos explorado:

### ✅ Lo que hemos cubierto:

1. **Carga del modelo** - Tanto en configuración básica como cuantizada
2. **Capacidades básicas** - Generación de texto general
3. **Capacidades conversacionales** - Formato de chat estructurado
4. **Capacidades multilingües** - Soporte para múltiples idiomas
5. **Generación de código** - Programación asistida por IA
6. **Análisis de rendimiento** - Métricas de velocidad y memoria
7. **Configuraciones de generación** - Parámetros de creatividad y coherencia

### 🎯 Puntos clave aprendidos:

- **Llama 3.1** es un modelo muy capaz para múltiples tareas
- La **cuantización** puede reducir significativamente el uso de memoria
- Los **parámetros de generación** afectan la creatividad vs coherencia
- El modelo maneja bien **múltiples idiomas** y **generación de código**
- El **formato de chat** permite conversaciones más naturales

### 🚀 Próximos pasos:

- Experimenta con diferentes prompts y configuraciones
- Prueba el modelo en tus propios casos de uso
- Explora técnicas de fine-tuning para tareas específicas
- Considera la implementación en aplicaciones reales

### 📚 Recursos adicionales:

- [Documentación de Transformers](https://huggingface.co/docs/transformers)
- [Llama 3.1 Model Card](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct)
- [Guías de optimización](https://huggingface.co/docs/transformers/perf_infer_gpu_one)

---

**¡Gracias por completar el Módulo 2 del Curso de Llama 3.1!** 🎉

In [12]:
# Limpieza final
print("\n🧹 Limpieza de memoria...")
if 'demo' in locals() and demo.model is not None:
    del demo.model
    del demo.tokenizer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    print("✅ Memoria liberada")

print("\n🎓 ¡Módulo 2 completado exitosamente!")
print("📝 Notebook guardado como: Modulo2_Llama31_Demo.ipynb")


🧹 Limpieza de memoria...
✅ Memoria liberada

🎓 ¡Módulo 2 completado exitosamente!
📝 Notebook guardado como: Modulo2_Llama31_Demo.ipynb
